## Random Forests

#### What is a Random Forest?
A Random Forest is an ensemble learning method that operates by constructing multiple decision trees during training and outputting the class that is the mode of the classes (for classification) or mean prediction (for regression) of the individual trees.

Bootstrapping:
* Each individual decision tree will be constructed on a bootstrapped subset of our data. If the dataset has n observations bootstrapping is the process of sampling n points with replacement which means that some obsverations in our data set will be selected more than once and some won't be selected at all. When we calculate that the probability an observation is omitted from our bootstrapped dataset is approximately 1/3
* Out-Of-Bag: The OOB samples are the ≈1/3 observations that were not selected to build a parcticular tree. 

In addition to bootstrapping, Random Forests introduce extra randomness when growing trees:
* At each node split, only a random subset of features is considered. This process is called "feature bagging" or "random subspace method"


#### Prediction using Random Forests:
For classification: Each tree "votes" for a class, and the majority vote wins
For regression: The average prediction of all trees is used

### Implementation

In [2]:
# Imports
import random
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Handle missing values
df.loc[df['Age'].isnull(),'Age'] = np.round(df['Age'].mean())
df.loc[df['Embarked'].isnull(),'Embarked'] = df['Embarked'].value_counts().index[0]

In [5]:
# Train, test splits
features = ['Pclass','Sex','Age','SibSp','Parch', 'Fare', 'Embarked']
nb_train = int(np.floor(0.9 * len(df)))
df = df.sample(frac=1, random_state=217)
X_train = df[features][:nb_train]
y_train = df['Survived'][:nb_train].values
X_test = df[features][nb_train:]
y_test = df['Survived'][nb_train:].values

### How to build decision tree
There are number of ways to build decision trees, most commonly used ones are Entropy, Classification Error, and Gini index.

### Entropy
-  a measurement of uncertainty (sometimes called impurity)
-  Certainity: Entropy is minimised when all samples in the node belong to the same class 
-  Uncertainity: Entropy is maximised when we have uniform distribution
  
### Information Gain
- Information gain measures how much information we gained when splitting a node at a particular value.


### Goal: Minimise the entropy and inturn maximise the information gain

In [6]:
def entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return -(p * np.log2(p) + (1-p) * np.log2(1-p))

def information_gain(left_subtree, right_subtree):
    parent = left_subtree + right_subtree
    p_parent = parent.count(1) /len(parent) if len(parent) > 0 else 0
    p_left = left_subtree.count(1) / len(left_subtree) if len(left_subtree) > 0 else 0
    p_right = right_subtree.count(1) / len(right_subtree) if len(right_subtree) > 0 else 0
    IG_p = entropy(p_parent)
    IG_left = entropy(p_left)
    IG_right = entropy(p_right)
    return IG_p - len(left_subtree)/len(parent) * IG_left - len(right_subtree)/ len(parent) * IG_right

In [7]:
def draw_bootstrap(x_train, y_train):
    bootstrap_ind = list(np.random.choice(range(len(x_train)),len(x_train), replacement=True))
    oob_ind = [i for i in range(len(x_train)) if i not in bootstrap_ind]
    x_bootstrap = x_train.iloc[bootstrap_ind].values
    y_bootstrap = y_train[bootstrap_ind]
    x_oob = x_train.iloc[oob_ind].values
    y_oob = y_train[oob_ind]
    return x_bootstrap, y_bootstrap, x_oob, y_oob

def oob_score(tree, x_test, y_test):
    miss_label = 0
    for i in range(len(x_test)):
        pred = predict_tree(tree, x_test[i])
        if pred != y_test[i]:            
            miss_label +=1
    return miss_label/len(x_test)